In [16]:
#!pip3 uninstall fitz -y
# !pip3 install frontend
#!pip3 install PyPDF2
!pip3 install PyMuPDF

     |████████████████████████████████| 4.1 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 30.1 MB 19.5 MB/s eta 0:00:01


In [11]:

# importing required modules 
import PyPDF2 
 
# creating a pdf file object 
pdfFileObj = open('Aug 2021 to oct 2021.pdf', 'rb') 
 
# creating a pdf reader object 
pdfReader = PyPDF2.PdfReader(pdfFileObj)
# creating a page object 
#pageObj = pdfReader.getPage(0) 
 
# extracting text from page 
#print(pageObj.extractText()) 
 
# closing the pdf file object 
pdfFileObj.close()

In [12]:
pdfReader

In [25]:
import fitz  # PyMuPDF

# Open the PDF file
pdf_path = 'Aug 2021 to oct 2021.pdf'
pdf_document = fitz.open(pdf_path)


# Read the text from each page and concatenate it into a single string
pdf_text = ""
for page_num in range(len(pdf_document)):
    page = pdf_document[page_num]
    pdf_text += page.get_text()

# Close the PDF after reading
pdf_document.close()

# Display the first 500 characters to get a sense of the data
print(pdf_text[:100000])

e-FILING REPORT COVER SHEET 
COMPANY NAME: 
DOES REPORT CONTAIN CONFIDENTIAL INFORMATION?  
No   
Yes  If yes, submit a redacted 
public version (or a cover letter) by email.  Submit the confidential information as directed in OAR 860-001-
0070 or the terms of an applicable protective order. 
Select report type:  
RE (Electric)   
RG (Gas)   
RW (Water)   
RT (Telecommunications)    
RO (Other, for example, industry safety information)    
Did you previously file a similar report?  
No 
Yes, report docket number:  
Report is required by: 
OAR 
Statute 
Order 
Note: A one-time submission required by an order is a compliance filing and not a report 
(file compliance in the applicable docket) 
Other 
(For example, federal regulations, or requested by Staff) 
Is this report associated with a specific docket/case?  
No 
Yes, docket number:  
List Key Words for this report.  We use these to improve search results.  
Send the completed Cover Sheet and the Report in an email addressed to PUC.F

In [28]:
import re
import pandas as pd

# Function to extract disconnection data
def extract_disconnections(text, month):
    pattern = f'{month}\n(.*?)Grand Total'
    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        # Extract zip codes and counts for the given month
        return dict(re.findall(r'(\d{5})\n(\d+)', matches[0]))
    return {}

# Extract disconnection data for each month
disconnection_data_aug = extract_disconnections(pdf_text, 'Aug 2021')
disconnection_data_sep = extract_disconnections(pdf_text, 'Sep 2021')
disconnection_data_oct = extract_disconnections(pdf_text, 'Oct 2021')

# Function to create a DataFrame from the disconnection data
def create_dataframe(disconnection_data):
    # Create a DataFrame from the disconnection data dictionary
    df = pd.DataFrame(list(disconnection_data.items()), columns=['ZipCode', 'Count'])
    df['ZipCode'] = df['ZipCode'].astype(int)  # Convert ZipCode to integer
    df['Count'] = df['Count'].astype(int)  # Convert Count to integer
    return df

# Create DataFrames for each month
df_aug = create_dataframe(disconnection_data_aug)
df_sep = create_dataframe(disconnection_data_sep)
df_oct = create_dataframe(disconnection_data_oct)

# Merge the DataFrames on ZipCode
merged_df = df_aug.merge(df_sep, on='ZipCode', how='outer', suffixes=('_Aug', '_Sep'))
merged_df = merged_df.merge(df_oct, on='ZipCode', how='outer')
merged_df.rename(columns={'Count': 'Count_Oct'}, inplace=True)

# Replace NaN values with 0 (assuming NaN means no disconnections)
merged_df.fillna(0, inplace=True)

# Convert counts to integers
merged_df['Count_Aug'] = merged_df['Count_Aug'].astype(int)
merged_df['Count_Sep'] = merged_df['Count_Sep'].astype(int)
merged_df['Count_Oct'] = merged_df['Count_Oct'].astype(int)

print(merged_df)


Empty DataFrame
Columns: [Count_Aug, Count_Sep, ZipCode, Count_Oct]
Index: []


In [29]:
import re
import pandas as pd

# Define the regex pattern to match zip codes and counts
zipcode_pattern = r'(\d{5})\s+(\d{1,7})\s+(\d{1,7})\s+(\d{1,7})'
disconnections_pattern = r'Data Point B - Number of\s+Disconnects by Zip Code(.*?)Data Point C'

# Find all matches for zip codes
zipcode_matches = re.findall(zipcode_pattern, pdf_text, re.DOTALL)

# Find all matches for disconnections
disconnections_matches = re.findall(disconnections_pattern, pdf_text, re.DOTALL)

# Parse disconnections data for each month
disconnections_data = {'Aug-21': {}, 'Sep-21': {}, 'Oct-21': {}}
months = ['Aug-21', 'Sep-21', 'Oct-21']
for month, match in zip(months, disconnections_matches):
    month_data = re.findall(r'(\d{5})\s+(\d+)', match)
    for zip_code, count in month_data:
        disconnections_data[month][zip_code] = int(count)

# Convert zip code matches to DataFrame
zipcode_df = pd.DataFrame(zipcode_matches, columns=['ZipCode', 'Active_Aug-21', 'Active_Sep-21', 'Active_Oct-21'])

# Convert disconnections data to DataFrame
disconnections_df_list = []
for month, data in disconnections_data.items():
    month_df = pd.DataFrame(list(data.items()), columns=['ZipCode', month])
    disconnections_df_list.append(month_df)

# Merge the disconnections DataFrames on ZipCode
merged_disconnections_df = disconnections_df_list[0]
for df in disconnections_df_list[1:]:
    merged_disconnections_df = pd.merge(merged_disconnections_df, df, on='ZipCode', how='outer')

# Merge the active customer counts with disconnection counts
final_df = pd.merge(zipcode_df, merged_disconnections_df, on='ZipCode', how='left')

# Replace NaN values with 0 (assuming NaN means no disconnections)
final_df.fillna(0, inplace=True)

# Convert all counts to integers
final_df.iloc[:, 1:] = final_df.iloc[:, 1:].astype(int)

# Display the final DataFrame
print(final_df)


    ZipCode  Active_Aug-21  Active_Sep-21  Active_Oct-21  Aug-21  Sep-21  \
0     97011            499            499            498       0       0   
1     97028            723            724            724       0       0   
2     97036              2              2              2       0       0   
3     97056             28             29             29       0       0   
4     97325            150            150            150       0       0   
..      ...            ...            ...            ...     ...     ...   
111   97301              1          97303              1       0       0   
112   97006              1          97003              1       0       0   
113   97062              2          97006              3       0       0   
114   97027              1          97045              1       0       0   
115   97301              1          97303              1       0       0   

     Oct-21  
0         0  
1         0  
2         0  
3         0  
4         0  
.. 

In [33]:
final_df.tail(50)

,ZipCode,Active_Aug-21,Active_Sep-21,Active_Oct-21,Aug-21,Sep-21,Oct-21
66,97305,3,97089,2,0,0,0
67,97115,1,97317,2,0,0,0
68,97101,1,97119,1,0,0,0
69,97392,1,97113,2,0,0,0
70,97114,2,97124,26,0,0,0
71,97123,9,97127,5,0,0,0
72,97124,6,97132,9,0,0,0
73,97128,1,97133,1,0,0,0
74,97132,3,97140,2,0,0,0
75,97137,1,97148,2,0,0,0


In [38]:
import re
import pandas as pd

# Assuming `pdf_text` contains the text extracted from the PDF

# Regular expression to match zip codes and the following numbers
zipcode_pattern = re.compile(
    r'(\d{5})\s+'               # Match the zip code
    r'(\d+)\s+(\d+)\s+(\d+)\s+' # Match active account numbers for each month
    r'(\d+)\s+(\d+)\s+(\d+)'    # Match disconnection numbers for each month
)

# Find all matches in the text
matches = zipcode_pattern.findall(pdf_text)

# Create a DataFrame from the matches
df = pd.DataFrame(matches, columns=[
    'Zip Code', 
    'Active Accounts Aug', 'Active Accounts Sep', 'Active Accounts Oct',
    'Disconnections Aug', 'Disconnections Sep', 'Disconnections Oct'
])

# Convert the string numbers to integers
df[['Active Accounts Aug', 'Active Accounts Sep', 'Active Accounts Oct',
    'Disconnections Aug', 'Disconnections Sep', 'Disconnections Oct']] = df[[
    'Active Accounts Aug', 'Active Accounts Sep', 'Active Accounts Oct',
    'Disconnections Aug', 'Disconnections Sep', 'Disconnections Oct'
]].astype(int)

# Display the DataFrame
df


,Zip Code,Active Accounts Aug,Active Accounts Sep,Active Accounts Oct,Disconnections Aug,Disconnections Sep,Disconnections Oct
0,97006,1,97002,1,97002,1,97007
1,97003,9,97003,11,97009,2,97005
2,97004,1,97013,1,97006,7,97005
3,97027,1,97007,3,97006,16,97028
4,97008,3,97007,6,97030,1,97009
5,97008,7,97035,1,97013,1,97009
6,97045,1,97015,6,97015,14,97068
7,97019,2,97022,1,97078,1,97020
8,97023,5,97113,1,97022,2,97024
9,97114,1,97024,11,97027,5,97116


In [40]:
import re
import pandas as pd

# Regex pattern adjusted for debugging
pattern = re.compile(
    r'(\d{5})\s+'  # Zip code
    r'(\d+)\s+(\d+)\s+(\d+)\s+'  # Active accounts for Aug, Sep, Oct
    r'(\d+)\s+(\d+)\s+(\d+)'  # Disconnections for Aug, Sep, Oct
)

# Find all matches in the text
matches = pattern.findall(pdf_text)

# Check the raw matches
for match in matches:
    print(match)

# Assuming the matches have the correct structure, create the DataFrame
# If the above print statements do not show the correct structure, you need to adjust your regex pattern accordingly
if matches:
    df = pd.DataFrame(matches, columns=[
        'Zip Code',
        'Active Accounts Aug', 'Active Accounts Sep', 'Active Accounts Oct',
        'Disconnections Aug', 'Disconnections Sep', 'Disconnections Oct'
    ])
    df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric)
    print(df)
else:
    print("No matches found, check the regex pattern and the structure of the text.")


('97006', '1', '97002', '1', '97002', '1', '97007')
('97003', '9', '97003', '11', '97009', '2', '97005')
('97004', '1', '97013', '1', '97006', '7', '97005')
('97027', '1', '97007', '3', '97006', '16', '97028')
('97008', '3', '97007', '6', '97030', '1', '97009')
('97008', '7', '97035', '1', '97013', '1', '97009')
('97045', '1', '97015', '6', '97015', '14', '97068')
('97019', '2', '97022', '1', '97078', '1', '97020')
('97023', '5', '97113', '1', '97022', '2', '97024')
('97114', '1', '97024', '11', '97027', '5', '97116')
('97027', '3', '97030', '18', '97119', '1', '97030')
('97035', '3', '97123', '1', '97032', '1', '97038')
('97124', '2', '97034', '1', '97045', '15', '97127')
('97035', '1', '97049', '1', '97132', '2', '97038')
('97055', '6', '97133', '1', '97042', '1', '97060')
('97203', '1', '97045', '8', '97062', '7', '97217')
('97055', '2', '97067', '1', '97224', '1', '97060')
('97068', '2', '97233', '8', '97062', '2', '97070')
('97236', '9', '97068', '2', '97071', '9', '97266')
('9707

In [41]:
df


,Zip Code,Active Accounts Aug,Active Accounts Sep,Active Accounts Oct,Disconnections Aug,Disconnections Sep,Disconnections Oct
0,97006,1,97002,1,97002,1,97007
1,97003,9,97003,11,97009,2,97005
2,97004,1,97013,1,97006,7,97005
3,97027,1,97007,3,97006,16,97028
4,97008,3,97007,6,97030,1,97009
5,97008,7,97035,1,97013,1,97009
6,97045,1,97015,6,97015,14,97068
7,97019,2,97022,1,97078,1,97020
8,97023,5,97113,1,97022,2,97024
9,97114,1,97024,11,97027,5,97116


In [42]:
import re
import pandas as pd

# Regex pattern adjusted for debugging
pattern = re.compile(
    r'(\d{5})\s+'  # Zip code
    r'(\d+)\s+(\d+)\s+(\d+)\s+'  # Active accounts for Aug, Sep, Oct
    r'(\d+)\s+(\d+)\s+(\d+)'  # Disconnections for Aug, Sep, Oct
)

# Find all matches in the text
matches = pattern.findall(pdf_text)

# Check the raw matches
for match in matches:
    print(match)

# Assuming the matches have the correct structure, create the DataFrame
# If the above print statements do not show the correct structure, you need to adjust your regex pattern accordingly
if matches:
    df = pd.DataFrame(matches, columns=[
        'Zip Code',
        'Active Accounts Aug', 'Active Accounts Sep', 'Active Accounts Oct',
        'Disconnections Aug', 'Disconnections Sep', 'Disconnections Oct'
    ])
    df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric)
    print(df)
else:
    print("No matches found, check the regex pattern and the structure of the text.")


('97006', '1', '97002', '1', '97002', '1', '97007')
('97003', '9', '97003', '11', '97009', '2', '97005')
('97004', '1', '97013', '1', '97006', '7', '97005')
('97027', '1', '97007', '3', '97006', '16', '97028')
('97008', '3', '97007', '6', '97030', '1', '97009')
('97008', '7', '97035', '1', '97013', '1', '97009')
('97045', '1', '97015', '6', '97015', '14', '97068')
('97019', '2', '97022', '1', '97078', '1', '97020')
('97023', '5', '97113', '1', '97022', '2', '97024')
('97114', '1', '97024', '11', '97027', '5', '97116')
('97027', '3', '97030', '18', '97119', '1', '97030')
('97035', '3', '97123', '1', '97032', '1', '97038')
('97124', '2', '97034', '1', '97045', '15', '97127')
('97035', '1', '97049', '1', '97132', '2', '97038')
('97055', '6', '97133', '1', '97042', '1', '97060')
('97203', '1', '97045', '8', '97062', '7', '97217')
('97055', '2', '97067', '1', '97224', '1', '97060')
('97068', '2', '97233', '8', '97062', '2', '97070')
('97236', '9', '97068', '2', '97071', '9', '97266')
('9707

In [43]:
import re
import pandas as pd

# Regex pattern to match the structure of the data
# This pattern is designed to capture a zip code followed by three groups of active account numbers,
# and then by three groups of disconnection numbers, all separated by whitespace.
pattern = re.compile(
    r'(\d{5})\s+'                  # Zip code
    r'(\d+)\s+(\d+)\s+(\d+)\s+'    # Active accounts for Aug, Sep, Oct
    r'(\d+)\s+(\d+)\s+(\d+)'       # Disconnections for Aug, Sep, Oct
)

# Find all matches in the text
matches = pattern.findall(pdf_text)

# If the length of the first match is not 7, it means our regex is not capturing correctly.
# We should check the first match to see what's being captured.
if matches and len(matches[0]) == 7:
    # Create a DataFrame from the matches
    df = pd.DataFrame(matches, columns=[
        'Zip Code',
        'Active Accounts Aug', 'Active Accounts Sep', 'Active Accounts Oct',
        'Disconnections Aug', 'Disconnections Sep', 'Disconnections Oct'
    ])

    # Convert the string numbers to integers
    df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric)

    # Print the DataFrame to verify correct parsing
    print(df)
else:
    print("The regex pattern did not match the data correctly. Here are the first few matches for review:")
    print(matches[:5])


   Zip Code Active Accounts Aug Active Accounts Sep Active Accounts Oct  \
0     97006                   1               97002                   1   
1     97003                   9               97003                  11   
2     97004                   1               97013                   1   
3     97027                   1               97007                   3   
4     97008                   3               97007                   6   
5     97008                   7               97035                   1   
6     97045                   1               97015                   6   
7     97019                   2               97022                   1   
8     97023                   5               97113                   1   
9     97114                   1               97024                  11   
10    97027                   3               97030                  18   
11    97035                   3               97123                   1   
12    97124              